# Exercise 4: Time series

Aim: To work with mooring data and plot time series

- Author: XX YOUR NAME XX
- Purpose: Plot velocity data
- Date: YYYY-MM-DD

At least four figures should be generated.

Choose four to make them show:
- the cosine fit to the CTD data and the residuals
- the cosine fit to the velocity data and the residuals
- the effect of filtering a time series
- a tidal ellipse

Save them according to the filenaming convention (check previous exercises for how to save a figure).
<hr>

In [ ]:
# Importing required packages here
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import os
import scipy.signal as sg
from scipy.optimize import curve_fit

import gsw


In [ ]:
# Set some paths
# Root directory - change this to the directory where you are working.
rootdir = '/Users/eddifying/Cloudfree/gitlab-cloudfree/messfern-plot-timeseries/'
#rootdir = 'C:\\Users\\Edward\\Documents\\GitHub\\messfern-plot-velo\\'
os.chdir(rootdir)
datadir = rootdir + 'data/'

# Output file paths for figures
figdir = '../figures/'
if not os.path.exists(figdir):
    os.makedirs(figdir)

# Load the CTD data

In [ ]:
# Specify the input file
inputfile = os.path.join(datadir, 'mooredCTD1_raw.nc')

# Load the input file
ctd_data = xr.open_dataset(inputfile)
# print(ctd_data)

# Plot the time series of PSAL
plt.figure(figsize=(15, 5))
plt.plot(ctd_data.time, ctd_data.PSAL, label=f'Mooring 1')

plt.xlabel('Time')
plt.ylabel('Salinity (PSU)')
plt.title('Time Series of Salinity (PSAL)')
plt.legend()
plt.grid(True)
plt.show()



In [ ]:
# Compute SA from PSAL and CT from TEMP
ctd_data['SA'] = gsw.SA_from_SP(ctd_data.PSAL, ctd_data.PRES, ctd_data.LON, ctd_data.LAT)
ctd_data['CT'] = # Add your code here

# plot the time series of SA and PSAL from mooring 1
plt.figure(figsize=(15, 5))
plt.plot(ctd_data.time, ctd_data.SA, label='SA')
plt.plot(ctd_data.time, ctd_data.PSAL, label='PSAL')
plt.xlabel('Time')
plt.ylabel('Salinity (PSU)')
plt.title('Time Series of Salinity (PSAL) and Absolute Salinity (SA) from Mooring 1')
plt.legend()
plt.grid(True)
# Calculate the 2.5th and 97.5th percentiles for PSAL and SA
psal_2_5 = ctd_data.PSAL.quantile(0.005)
psal_97_5 = ctd_data.PSAL.quantile(0.995)
sa_2_5 = ctd_data.SA.quantile(0.005)
sa_97_5 = ctd_data.SA.quantile(0.995)

# Set y-axis limits to the middle 95% of the PSAL and SA values
plt.ylim(np.floor(min(psal_2_5, sa_2_5)) - 0.1, np.ceil(max(psal_97_5, sa_97_5)) + 0.1)

plt.show()



# Fit a tidal harmonic

Since the region has strong tidal variability, let's see if we can extract the tidal cycle

In [ ]:
# Extract time in days since the start
time_in_days1 = (ctd_data.time - ctd_data.time[0]).astype('timedelta64[s]') / np.timedelta64(1, 's') / 86400
time_in_days2 = (ctd_data.time - ctd_data.time[0]).astype('timedelta64[D]') / np.timedelta64(1, 'D')

print(f"Min of time_in_days1: {time_in_days1.min().values}")
print(f"Max of time_in_days1: {time_in_days1.max().values}")

print(f"Time interval: {time_in_days1.min().values} to {time_in_days1.max().values}")
time_diff = np.diff(time_in_days1) 
print(time_diff)

# Turn on this plot to see how the time differs between time_in_days1 and time_in_days2
if 0:
    # Plot the two time representations against the original time in separate subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

    # Plot time in days (seconds conversion)
    ax1.plot(ctd_data.time, time_in_days1, label='Time in days (seconds conversion)')
    ax1.set_ylabel('Time in Days (seconds conversion)')
    ax1.set_title('Time in Days (seconds conversion)')
    ax1.legend()
    ax1.grid(True)

    # Plot time in days (days conversion)
    ax2.plot(ctd_data.time, time_in_days2, label='Time in days (days conversion)')
    ax2.set_xlabel('Original Time')
    ax2.set_ylabel('Time in Days (days conversion)')
    ax2.set_title('Time in Days (days conversion)')
    ax2.legend()
    ax2.grid(True)

    plt.show()


# Fitting a harmonic

Read about [curve_fit](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) from the `scipy.optimize` library: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

In [ ]:
# Define the cosine function with a period of the M2 lunar tide (12.42 hours)
def semi_diurnal_cosine(t, amplitude, phase, offset):
    period = 12.42 / 24  # Convert hours to days
    return amplitude * np.cos(2 * np.pi * t / period + phase) + offset

# Extract salinity data for a specific mooring (e.g., Mooring 1)
salinity_data = ctd_data.PSAL.values

# Choose which of the two time_in_days above to use
time_in_days = time_in_days1

# Fit the semi-diurnal cosine function to the salinity data
# Remove points which are not finite from the time_in_days and salinity_data vectors
finite_indices = np.isfinite(time_in_days) & np.isfinite(salinity_data)
time_in_days = time_in_days[finite_indices]
salinity_data = salinity_data[finite_indices]

# Remove salinity values below 19
valid_indices = salinity_data >= 19
time_in_days = time_in_days[valid_indices]
salinity_data = salinity_data[valid_indices]

# Read about scipy.optimize curve_fit function https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html
popt, pcov = curve_fit(semi_diurnal_cosine, time_in_days, salinity_data, p0=[1, 0, np.mean(salinity_data)])

# Extract the fitted parameters
amplitude, phase, offset = popt

# Generate the fitted salinity values
fitted_salinity = semi_diurnal_cosine(time_in_days, amplitude, phase, offset)

# Turn on this plot to see the fit vs the original data
if 0:
    # Plot the original and fitted salinity data
    plt.figure(figsize=(15, 5))
    plt.plot(time_in_days, salinity_data, label='Original Salinity Data')
    plt.plot(time_in_days, fitted_salinity, label='Fitted Semi-Diurnal Cosine', linestyle='--')
    plt.xlabel('Time (days)')
    plt.ylabel('Salinity (PSU)')
    plt.title('Fitting Semi-Diurnal Cosine to Salinity Data')
    plt.legend()
    plt.grid(True)
    plt.show()


# Load the velocity time series

In [ ]:
# Define the path to the file
velo_file = os.path.join(datadir, 'mooring1velocity.nc')

# Open the dataset
velo_data = xr.open_dataset(velo_file)

# You can print a list of variables to see what they're called
print(list(velo_data.data_vars))

# Change the 0 to 1 to turn on the plotting
if 0:
    # Plot the time series of UVEL, VVEL, and WVEL
    plt.figure(figsize=(15, 10))

    # Plot UVEL
    plt.subplot(3, 1, 1)
    plt.plot(velo_data.time, velo_data.UVEL, label='UVEL')
    plt.xlabel('Time')
    plt.ylabel('U Velocity (m/s)')
    plt.title('Time Series of U Velocity (UVEL)')
    plt.legend()
    plt.grid(True)

    # Plot VVEL
    plt.subplot(3, 1, 2)
    plt.plot(velo_data.time, velo_data.VVEL, label='VVEL')
    plt.xlabel('Time')
    plt.ylabel('V Velocity (m/s)')
    plt.title('Time Series of V Velocity (VVEL)')
    plt.legend()
    plt.grid(True)

    # Plot WVEL
    plt.subplot(3, 1, 3)
    # Add a line here to print the vertical velocity data
    plt.xlabel('Time')
    plt.ylabel('W Velocity (m/s)')
    plt.title('Time Series of W Velocity (WVEL)')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()



In [ ]:
# Extract UVEL and WVEL data
uvel_data = velo_data.UVEL.values
wvel_data = velo_data.WVEL.values

# Use the same time_in_days as before
time_in_days = (velo_data.time - velo_data.time[0]).astype('timedelta64[s]') / np.timedelta64(1, 's') / 86400
# Extract UVEL and VVEL data
uvel_data = velo_data.UVEL.values
vvel_data = velo_data.VVEL.values

# Use the same time_in_days as before
time_in_days = (velo_data.time - velo_data.time[0]).astype('timedelta64[s]') / np.timedelta64(1, 's') / 86400

# Remove points which are not finite from the time_in_days and velocity data vectors
finite_indices_uvel = np.isfinite(time_in_days) & np.isfinite(uvel_data)
finite_indices_vvel = np.isfinite(time_in_days) & np.isfinite(vvel_data)

time_in_days_uvel = time_in_days[finite_indices_uvel]
uvel_data = uvel_data[finite_indices_uvel]

time_in_days_vvel = time_in_days[finite_indices_vvel]
vvel_data = vvel_data[finite_indices_vvel]

# Fit the semi-diurnal cosine function to the UVEL data
popt_uvel, _ = curve_fit(semi_diurnal_cosine, time_in_days_uvel, uvel_data, p0=[1, 0, np.mean(uvel_data)])
amplitude_uvel, phase_uvel, offset_uvel = popt_uvel
fitted_uvel = semi_diurnal_cosine(time_in_days_uvel, amplitude_uvel, phase_uvel, offset_uvel)

# Fit the semi-diurnal cosine function to the WVEL data
popt_vvel, _ = curve_fit(semi_diurnal_cosine, time_in_days_vvel, vvel_data, p0=[1, 0, np.mean(vvel_data)])
amplitude_vvel, phase_vvel, offset_vvel = popt_vvel
fitted_vvel = semi_diurnal_cosine(time_in_days_vvel, amplitude_vvel, phase_vvel, offset_vvel)

# Change the 0 to 1 to turn on the plot
if 0:
    # Plot the original and fitted UVEL data
    plt.figure(figsize=(15, 10))

    plt.subplot(2, 1, 1)
    plt.plot(time_in_days_uvel, uvel_data, label='Original UVEL Data')
    plt.plot(time_in_days_uvel, fitted_uvel, label='Fitted Semi-Diurnal Cosine', linestyle='--')
    plt.xlabel('Time (days)')
    plt.ylabel('U Velocity (m/s)')
    plt.title('Fitting Semi-Diurnal Cosine to U Velocity (UVEL)')
    plt.legend()
    plt.grid(True)

    # Plot the original and fitted WVEL data
    plt.subplot(2, 1, 2)
    plt.plot(time_in_days_vvel, vvel_data, label='Original VVEL Data')
    plt.plot(time_in_days_vvel, fitted_vvel, label='Fitted Semi-Diurnal Cosine', linestyle='--')
    plt.xlabel('Time (days)')
    plt.ylabel('W Velocity (m/s)')
    plt.title('Fitting Semi-Diurnal Cosine to V Velocity (WVEL)')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Plotting a tidal ellipse

Add the original data to the plot as a scatter (see `ax1.scatter()`)

In [ ]:
# Generate time values for one period (12.42 hours)
period_hours = 12.42
period_days = period_hours / 24
time_values = np.linspace(0, period_days, 1000)

# Generate fitted uvel and vvel values using the fitted parameters
fitted_uvel_values = semi_diurnal_cosine(time_values, amplitude_uvel, phase_uvel, offset_uvel)
fitted_vvel_values = semi_diurnal_cosine(time_values, amplitude_vvel, phase_vvel, offset_vvel)

# Plot the ellipse and the polar plot
fig, ax1 = plt.subplots(figsize=(16, 8))

# Plot the ellipse with square axes
ax1.plot(fitted_uvel_values, fitted_vvel_values, label='Fitted Ellipse')
ax1.set_xlabel('U Velocity (m/s)')
ax1.set_ylabel('V Velocity (m/s)')
ax1.set_title('Ellipse Generated by Harmonic Fit to UVEL and VVEL (Square Axes)')
ax1.legend()
ax1.grid(True)
ax1.axis('equal')


plt.show()

# Filter data

Sometimes we would like to low-pass filter (remove the high frequency variability) by using a running average (also called a moving average, or convolving with a boxcar filter).  See below for examples.

In [ ]:
# Apply a 1-day boxcar filter to the data variables SA and CT
# Calculate the number of samples in 24 hours
#time_diff_seconds = (ctd_data.time[1] - ctd_data.time[0]).values / np.timedelta64(1, 's')
time_diff = np.diff(ctd_data.time.values) / np.timedelta64(1, 's')
time_diff_seconds = np.mean(time_diff)
print(f"Average time difference in seconds: {time_diff_seconds}")

print(f"Time difference in seconds: {time_diff_seconds}")
samples_per_day = int(24 * 3600 / time_diff_seconds)
print(f"Number of samples in 24 hours: {samples_per_day}")

Npoints = samples_per_day
Npoints = int(Npoints)
time_filtered = time_in_days1.rolling(time=Npoints, center=True).mean()
ctd_data['SA_filtered'] = ctd_data.SA.rolling(time=Npoints, center=True).mean()

# This plot is to check how the original and filtered time vectors look
if 0: 
    # Plot the original and filtered time series of time_in_days1
    plt.figure(figsize=(15, 5))

    plt.plot(ctd_data.time, time_in_days1, label='Original Time')
    plt.plot(ctd_data.time, time_filtered, label='Filtered Time', linestyle='--')

    plt.xlabel('Time')
    plt.ylabel('Time in Days')
    plt.title('Original and Filtered Time vector')
    plt.legend()
    plt.grid(True)

    plt.show()


In [ ]:
# Plot the original and filtered SA time series. 

- You will need to add a line of code to do this

In [ ]:
# Plot the original and filtered time series of SA and CT from mooring 1
plt.figure(figsize=(15, 10))

# Plot SA
plt.subplot(2, 1, 1)
plt.plot(ctd_data.SA, label='Original SA')
#
# Add your code here--- plot the filtered SA
#
# 
plt.xlabel('Time')
plt.ylabel('Absolute Salinity (SA) (PSU)')
plt.title('Time Series of Absolute Salinity (SA) from Mooring 1')
plt.legend()
plt.grid(True)

plt.ylim(19, 30)

plt.tight_layout()
plt.show()


# Verify that the filter is doing what you expect

In [ ]:
# Manually average the data for a speciic date

# Define the time range
start_date = '2024-05-22'
end_date = '2024-05-22'

# Select the data within the specified time range for mooring=1
sa_data_range = ctd_data.SA.sel(time=slice(start_date, end_date))
SA_filtered_range = ctd_data.SA_filtered.sel(time=slice(start_date, end_date))

# Manual average of SA
average_sa = sa_data_range.mean().values

# Plot the Absolute Salinity (SA) data within the specified time range
plt.figure(figsize=(15, 5))
plt.plot(sa_data_range.time, sa_data_range, label=f'Mooring 1')
plt.axhline(y=average_sa, color='r', linestyle='--', label='Average SA')
plt.plot(SA_filtered_range.time, SA_filtered_range, linestyle='--', color='green', label='Filtered SA')
plt.xlabel('Time')
plt.ylabel('Absolute Salinity (SA) (PSU)')
plt.title(f'Time Series of Absolute Salinity (SA) between {start_date} and {end_date}')
plt.legend()
plt.grid(True)
plt.show()

# If correct, average SA (the horizontal line) should intersect the filtered SA (the green line) at the midpoint of the day.



## You can also extract the data at the midpoint of the day

This code shows how to use time manipulation to create noon on the day "start_date" and then to find the nearest point in time from the time vector to that specific time.


In [ ]:
# From above, print the average value we've determined
print(f"Average Absolute Salinity (SA): {average_sa}")

# Create the time for noon on the date of start_date
specific_time = np.datetime64(start_date) + np.timedelta64(12, 'h')
# Find the value of SA_filtered nearest in time to the specific time
nearest_time_index = np.abs(ctd_data.time - specific_time).argmin()
# Get the value at this index (where the index is the location within the vector)
nearest_sa_filtered_value = ctd_data.SA_filtered.isel(time=nearest_time_index).values

print(f"Nearest SA_filtered value at {specific_time}: {nearest_sa_filtered_value}")

# Check how well the two quantities match.
# NOTE! if you have changed the filtering above (away from being 1-day boxcar) you may get a differen tanswer.

## Optional

### In the following, you can edit the code to try out different filters

In [ ]:
Npoints = int(samples_per_day/2) # 12 hours

# Define the Tukey window parameter (alpha)
alpha = 0.5

# Create the Tukey window
tukey_window = sg.windows.tukey(Npoints, alpha)

# A moving average is also called a boxcar filter
boxcar_window = sg.windows.boxcar(Npoints)


# To apply the filtering window, "convolve" the window with the original data
# Apply the Boxcar window to the SA data
ctd_data['SA_boxcar_filtered'] = xr.DataArray(
    np.convolve(ctd_data.SA, boxcar_window, mode='same') / boxcar_window.sum(),
    dims='time',
    coords={'time': ctd_data.time}
)
# Apply the Tukey window to the SA data
ctd_data['SA_tukey_filtered'] = xr.DataArray(
    np.convolve(ctd_data.SA, tukey_window, mode='same') / tukey_window.sum(),
    dims='time',
    coords={'time': ctd_data.time}
)

# Define the time range for zooming in
start_zoom_date = '2024-05-22'
end_zoom_date = '2024-05-22'

# Select the data within the specified time range
sa_zoom_range = ctd_data.SA.sel(time=slice(start_zoom_date, end_zoom_date))
sa_boxcar_zoom_range = ctd_data.SA_boxcar_filtered.sel(time=slice(start_zoom_date, end_zoom_date))
sa_tukey_zoom_range = ctd_data.SA_tukey_filtered.sel(time=slice(start_zoom_date, end_zoom_date))

# Plot the zoomed-in time series of SA from mooring 1
plt.figure(figsize=(15, 10))

# Plot SA
plt.subplot(2, 1, 1)
plt.plot(sa_zoom_range.time, sa_zoom_range, label='Original SA')
plt.plot(sa_boxcar_zoom_range.time, sa_boxcar_zoom_range, label='Boxcar Filtered SA')
plt.plot(sa_tukey_zoom_range.time, sa_tukey_zoom_range, label='Tukey Filtered SA')
plt.xlabel('Time')
plt.ylabel('Absolute Salinity (SA) (PSU)')
plt.title('Zoomed Time Series of Absolute Salinity (SA) from Mooring 1 on 2024-05-22')
plt.legend()
plt.grid(True)

plt.ylim(26, 30)

plt.tight_layout()
plt.show()

In [ ]:
# Defining another function to apply Tukey and Boxcar filters to the given time series data
# Here, window_size is given a default value of 1440 (1 day) and alpha is given a default value of 0.5
def apply_filters(data, window_size=1440, alpha=0.5):
    """
    Apply Tukey and Boxcar filters to the given time series data.

    Parameters:
    data (numpy.ndarray or xarray.DataArray): The input time series data.
    window_size (int): The size of the window for the filters.
    alpha (float): The shape parameter for the Tukey window (default is 0.5).

    Returns:
    tuple: A tuple containing the Tukey filtered data and the Boxcar filtered data.
    """
    window_size=int(window_size)

    # Create the Tukey and Boxcar windows
    tukey_window = sg.windows.tukey(window_size, alpha)
    boxcar_window = sg.windows.boxcar(window_size)

    # Apply the Tukey window to the data
    tukey_filtered = np.convolve(data, tukey_window, mode='same') / tukey_window.sum()

    # Apply the Boxcar window to the data
    boxcar_filtered = np.convolve(data, boxcar_window, mode='same') / boxcar_window.sum()

    return tukey_filtered, boxcar_filtered

# Example usage:
# Assuming `ctd_data['SA']` is the input time series data
window_size = Npoints  # Use the predefined window size
alpha = 0.5  # Use the predefined alpha value
varname = 'SA'
tukey_filtered, boxcar_filtered = apply_filters(ctd_data[varname].values, window_size, alpha)

# Plot the original and filtered data
plt.figure(figsize=(15, 10))

# Plot original SA
plt.subplot(2, 1, 1)
plt.plot(ctd_data.time, ctd_data[varname], label='Original SA')
plt.plot(ctd_data.time, tukey_filtered, label='Tukey Filtered SA', linestyle='--')
plt.plot(ctd_data.time, boxcar_filtered, label='Boxcar Filtered SA', linestyle='--')
plt.xlabel('Time')
plt.ylabel('Absolute Salinity (SA) (PSU)')
plt.title('Original and Filtered Absolute Salinity (SA)')
plt.legend()
plt.grid(True)
plt.show()
